In [1]:
import sys
import os
from pathlib import Path
import pickle
import torch
import pprint
sys.path.append(os.getcwd())
sys.path.append(str(Path(os.getcwd()).parent))
from utils import *
from configuration import *
import numpy as np

if not os.path.exists('./evaluation/'):
    os.makedirs('./evaluation/')

In [2]:
args = parser.parse_args([])
args.task = 'temporal'
args.metric = 'nominal_snn'
args.SoftEva = True
args = FormulateArgs(args)

In [3]:
results = torch.zeros([15,10,2])

for seed in range(10):
    for ds in range(15):
        args.SEED = seed
        args.DATASET = ds

        valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')
        test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')

        modelname = f"baseline_model_SNN_data_{args.DATASET:02d}_{datainfo['dataname']}_seed_{args.SEED:02d}.model"
        print(modelname)
        
        if os.path.isfile(f'./models/{modelname}'):                
            model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
            # model.UpdateArgs(args)

            SetSeed(args.SEED)

            evaluator = Evaluator(args).to(args.DEVICE)

            for x,y in valid_loader:
                X_valid, y_valid = x.to(args.DEVICE), y.to(args.DEVICE)
            for x,y in test_loader:
                X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)

            acc_valid = evaluator(model, X_valid, y_valid)
            acc_test   = evaluator(model, X_test,  y_test)

            results[ds,seed,0] = acc_valid
            results[ds,seed,1] = acc_test
        else:
            results[ds,seed,:] = float('nan')

baseline_model_SNN_data_00_cbf_seed_00.model
baseline_model_SNN_data_01_distalphalanxtw_seed_00.model
baseline_model_SNN_data_02_freezerregulartrain_seed_00.model
baseline_model_SNN_data_03_freezersmalltrain_seed_00.model
baseline_model_SNN_data_04_gunpointagespan_seed_00.model
baseline_model_SNN_data_05_gunpointmaleversusfemale_seed_00.model
baseline_model_SNN_data_06_gunpointoldversusyoung_seed_00.model
baseline_model_SNN_data_07_middlephalanxoutlineagegroup_seed_00.model
baseline_model_SNN_data_08_mixedshapesregulartrain_seed_00.model
baseline_model_SNN_data_09_powercons_seed_00.model
baseline_model_SNN_data_10_proximalphalanxoutlinecorrect_seed_00.model
baseline_model_SNN_data_11_selfregulationscp2_seed_00.model
baseline_model_SNN_data_12_slope_seed_00.model
baseline_model_SNN_data_13_smoothsubspace_seed_00.model
baseline_model_SNN_data_14_symbols_seed_00.model
baseline_model_SNN_data_00_cbf_seed_01.model
baseline_model_SNN_data_01_distalphalanxtw_seed_01.model
baseline_model_SNN_d

In [4]:
results.shape

torch.Size([15, 10, 2])

In [5]:
N_selected_seeds = 3
re_temp = torch.nan_to_num(results, nan=-10000.)
values, indices = torch.topk(re_temp[:,:,0], N_selected_seeds, dim=1, largest=True, sorted=True)
selected_results = results.gather(1, indices.unsqueeze(-1).expand(-1, -1, 2))
selected_test_mean = selected_results[:,:,-1].mean(1)
selected_test_std = selected_results[:,:,-1].std(1)
selected_test_results = torch.cat([selected_test_mean.unsqueeze(-1), selected_test_std.unsqueeze(-1)], dim=1)

np.savetxt(f"./evaluation/top_{N_selected_seeds}_SNN_acc.txt", selected_test_results.numpy(), delimiter="\t", fmt='%.5f')